To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do:

- [data prep](#Data)
- [train](#Train) 
- [run the model](#Inference)


# Fin-Tune LLM to downstream task using Unsloth & TRL from HF

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.759 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
I will use the MLSUM, a large-scale MultiLingual SUMmarization dataset, [MLSUM](https://huggingface.co/datasets/mlsum).

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mlsum", "fr")
train_ds = dataset['train']
train_ds

In [ ]:
print(dataset['train'][2])

{'text': 'Un juge américain a rejeté, jeudi 31 décembre, les accusations portées contre cinq employés de Blackwater – la plus grande entreprise privée de sécurité utilisée par les Etats-Unis en Irak – accusés d\'avoir ouvert le feu à Bagdad sur la foule et tué quatorze Irakiens en 2007 lors d\'une mission pour le département d\'Etat. Une décision qui aussitôt suscité l\'indignation en Irak : "J\'ai été vraiment stupéfaite de la décision et j\'attends de l\'ambassade américaine une copie de la décision du juge", a ainsi affirmé la ministre des droits de l\'homme irakienne, Wejdane Mikhaïl. "Ce qui s\'est passé a été très mauvais, car tant de personnes innocentes sont mortes, des jeunes, des étudiants, tués par quelqu\'un qui aimait tirer sur des gens non armés." Le juge fédéral Ricardo Urbina a argué dans sa décision que l\'accusation avait violé les droits des accusés en utilisant des témoignages livrés alors que les agents de sécurité étaient protégés par l\'immunité que leur conférai

In [13]:
# create random resumer instruction 
resumer_requests = [
"Pourriez-vous résumer ce texte, s'il vous plaît?",
"Peux-tu faire un résumé de ce texte?",
"Je vous serais reconnaissant de bien vouloir résumer ce texte.",
"Pourrais-tu me donner les points principaux de ce texte?",
"Pouvons-nous avoir un bref résumé de ce texte?",
"Est-ce que tu pourrais condenser ce texte en quelques phrases clés?",
"Pourriez-vous m'expliquer brièvement ce que dit ce texte?",
"Peux-tu dégager les idées essentielles de ce texte?",
"Je vous prie de me fournir un résumé succinct de ce texte.",
"Pourriez-vous éclaircir les points principaux de ce texte en quelques phrases?"]

In [15]:
import random

# define the prompte template
resumer_prompt = """Voici une instruction qui décrit une tâche, accompagnée d'une entrée qui fournit un contexte supplémentaire. Rédigez une réponse qui complète correctement la demande.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [16]:
print(resumer_prompt.format("tata","toto","titi"))

Voici une instruction qui décrit une tâche, accompagnée d'une entrée qui fournit un contexte supplémentaire. Rédigez une réponse qui complète correctement la demande.

### Instruction:
tata

### Input:
toto

### Response:
titi


In [17]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# define a function for example sample formating
def formatting_prompts_func(examples):
    
    # create list of random instructions
    instructions = [random.choice(resumer_requests) for i in range(len(examples["text"]))]
    inputs       = examples["text"]
    outputs      = examples["summary"]

    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = resumer_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    
    examples['textinst'] =  texts
    return examples

#### apply to train DS

In [21]:
train_ds = train_ds.map(formatting_prompts_func, batched = True,)

In [22]:
len(train_ds)

392902

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    dataset_text_field = "textinst",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 3,
        warmup_steps = 5,
        num_train_epochs = 2,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        hub_model_id='medelharchaoui/bert-finetune',
        push_to_hub=True
    ),
)

In [24]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.759 GB.
2.344 GB of memory reserved.


In [25]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 3
\        /    Total batch size = 24 | Total steps = 4,166
 "-____-"     Number of trainable parameters = 29,884,416


  0%|          | 0/4166 [00:00<?, ?it/s]

{'loss': 1.9581, 'grad_norm': 0.16983595490455627, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 1.934, 'grad_norm': 0.2197076380252838, 'learning_rate': 8e-05, 'epoch': 0.0}
{'loss': 2.0231, 'grad_norm': 0.20478251576423645, 'learning_rate': 0.00012, 'epoch': 0.0}


KeyboardInterrupt: 

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
rand_test_smple = train_ds[1656]

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    resumer_prompt.format(
        random.choice(resumer_requests), # instruction
        rand_test_smple['text'], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

In [ ]:
result = tokenizer.batch_decode(outputs)[0]
print(result[result.find('### Response'):])

In [ ]:
rand_test_smple['summary']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    resumer_prompt.format(
        random.choice(resumer_requests), # instruction
        rand_test_smple['text'], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving

In [27]:
model.push_to_hub(repo_id='medelharchaoui/Phi-3-mini-4k-instruct-finetune-FR-Summarisation') # Online saving

adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

Saved model to https://huggingface.co/medelharchaoui/Phi-3-mini-4k-instruct-finetune-FR-Summarisation


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)